# Comparison of Models w/ Different Lambdas

In [ ]:
! hadoop fs -copyToLocal gs://isomantics/code/ /root/notebooks/code/

In [3]:
! hadoop fs -ls gs://isomantics/pickle/en_fasttext_vocab.pkl

17/10/12 07:14:29 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: 1.6.1-hadoop2
-rwx------   3 root root   23588804 2017-10-02 22:40 gs://isomantics/pickle/en_fasttext_vocab.pkl


In [20]:

! pip install keras
! pip install matplotlib
! pip install pandas
! pip install seaborn
! pip install numpy
! pip install tensorflow
! pip install pydrive --upgrade
! pip install sklearn


Requirement already up-to-date: oauth2client>=4.0.0 in /opt/conda/lib/python3.5/site-packages (from pydrive)
Requirement already up-to-date: PyYAML>=3.0 in /opt/conda/lib/python3.5/site-packages (from pydrive)
Requirement already up-to-date: google-api-python-client>=1.2 in /opt/conda/lib/python3.5/site-packages (from pydrive)
Requirement already up-to-date: pyasn1-modules>=0.0.5 in /opt/conda/lib/python3.5/site-packages (from oauth2client>=4.0.0->pydrive)
Requirement already up-to-date: httplib2>=0.9.1 in /opt/conda/lib/python3.5/site-packages (from oauth2client>=4.0.0->pydrive)
Requirement already up-to-date: six>=1.6.1 in /opt/conda/lib/python3.5/site-packages (from oauth2client>=4.0.0->pydrive)
Requirement already up-to-date: pyasn1>=0.1.7 in /opt/conda/lib/python3.5/site-packages (from oauth2client>=4.0.0->pydrive)
Requirement already up-to-date: rsa>=3.1.4 in /opt/conda/lib/python3.5/site-packages (from oauth2client>=4.0.0->pydrive)
Requirement already up-to-date: uritemplate<4de

In [11]:
y

NameError: name 'y' is not defined

In [42]:
! pip install --upgrade google-cloud-storage-2.14.2 -t /opt/conda/lib/python3.5/site-packages

  Could not find a version that satisfies the requirement google-cloud-storage-2.14.2 (from versions: )
No matching distribution found for google-cloud-storage-2.14.2


In [49]:
! pip install --upgrade google-cloud -t /opt/conda/lib/python3.5/site-packages

  Using cached google_cloud-0.27.0-py2.py3-none-any.whl
  Using cached google_cloud_datastore-1.2.0-py2.py3-none-any.whl
  Using cached google_cloud_speech-0.28.0-py2.py3-none-any.whl
  Using cached google_cloud_storage-1.3.2-py2.py3-none-any.whl
  Using cached google_cloud_dns-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_spanner-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_vision-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_core-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_runtimeconfig-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_language-0.27.0-py2.py3-none-any.whl
  Using cached google_cloud_monitoring-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_videointelligence-0.25.0-py2.py3-none-any.whl
  Using cached google_cloud_bigtable-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_logging-1.2.0-py2.py3-none-any.whl
  Using cached google_cloud_bigquery-0.26.0-py2.py3-none-any.whl
  Using cached google_cloud_resource

In [21]:
from keras import backend as K
from keras import regularizers
import nregularizer
import numpy as np
from gensim_download import pickle_rw
from keras.layers import Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
# import plotly.plotly as py
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy.linalg

from numpy.linalg import det

from numpy.linalg import inv

#import sympy as sympy

import pandas as pd
%matplotlib inline
import seaborn as sns; sns.set()

#import cloudstorage

#from sympy import Matrix
# from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot



ImportError: cannot import name 'scopes_to_string'

### Defining Functions

In [35]:
def make_dict(vocab, vectors):
    """Make dictionary of vocab and vectors"""
    return {vocab[i]: vectors[i] for i in range(len(vocab))}


def vocab_train_test(embedding, lg1, lg2, lg1_vocab):
    """Create training and test vocabularies"""
    if embedding == 'zeroshot':
        with open('../data/zeroshot/transmat/data/' +
                  'OPUS_en_it_europarl_train_5K.txt') as f:
            vocab_train = [(_.split(' ')[0], _.split(' ')[1])
                           for _ in f.read().split('\n')[:-1]]
        with open('../data/zeroshot/transmat/data/' +
                  'OPUS_en_it_europarl_test.txt') as f:
            vocab_test = [(_.split(' ')[0], _.split(' ')[1])
                          for _ in f.read().split('\n')[:-1]]

    elif embedding in ['fasttext_random', 'fasttext_top']:
        embedding, split = embedding.split('_')
        lg1_lg2, lg2_lg1 = pickle_rw((lg1 + '_' + lg2, 0),
                                     (lg2 + '_' + lg1, 0), write=False)
        # T = Translation, R = Reverse (translated and then translated back)
        # Create vocab from 2D translations
        vocab_2D = []
        for lg1_word in lg1_vocab:
            # Translate lg1_word
            if lg1_word in lg1_lg2:
                lg1_word_T = lg1_lg2[lg1_word]

                # Check if translated word (or lowercase) is in lg2_lg1
                if lg1_word_T in lg2_lg1.keys():
                    lg1_word_R = lg2_lg1[lg1_word_T]
                elif lg1_word_T.lower() in lg2_lg1.keys():
                    lg1_word_T = lg1_word_T.lower()
                    lg1_word_R = lg2_lg1[lg1_word_T]
                else:
                    lg1_word_R = None

                # Check if lg1_word and lg1_word_R are equal (lowercase)
                if lg1_word_R:
                    if lg1_word.lower() == lg1_word_R.lower():
                        vocab_2D.append((lg1_word, lg1_word_T))
        print('length of '+ lg1+'-'+ lg2+ ' vocab: '+str(len(vocab_2D)))

        #Create Train/Test vocab

        if split == 'random':
            sample = np.random.choice(len(vocab_2D), 6500, replace=False)
            vocab_train = np.asarray(vocab_2D)[sample[:5000]].tolist()
            vocab_test = np.asarray(vocab_2D)[sample[5000:]].tolist()
        elif split == 'top':
#             sample = np.random.choice(range(6500), 6500, replace=False)
#             vocab_train = np.asarray(vocab_2D)[:5000, :].tolist()
#             vocab_test = np.asarray(vocab_2D)[:1500, :].tolist()
            
#             sample = np.random.choice(range(6500), 6500, replace=False)
            vocab_train = np.asarray(vocab_2D)[:700, :].tolist()
            vocab_test = np.asarray(vocab_2D)[:200, :].tolist()
        else:
            pass

        # if split == 'random':
        #     sample = np.random.choice(len(vocab_2D), 900, replace=False)
        #     vocab_train = np.asarray(vocab_2D)[sample[:700]].tolist()
        #     vocab_test = np.asarray(vocab_2D)[sample[700:]].tolist()
        # elif split == 'top':
        #     sample = np.random.choice(range(900), 900, replace=False)
        #     vocab_train = np.asarray(vocab_2D)[:700, :].tolist()
        #     vocab_test = np.asarray(vocab_2D)[:200, :].tolist()
        # else:
        #     pass

    return vocab_train, vocab_test


def vectors_train_test(vocab_train, vocab_test):
    """Create training and test vectors"""
    X_train, y_train = zip(*[(lg1_dict[lg1_word], lg2_dict[lg2_word])
                             for lg1_word, lg2_word in vocab_train])
    X_test, y_test = zip(*[(lg1_dict[lg1_word], lg2_dict[lg2_word])
                           for lg1_word, lg2_word in vocab_test])
    return map(np.asarray, (X_train, X_test, y_train, y_test))




def translation_matrix(X_train, y_train, l):
    """Fit translation matrix T"""
    #def norm_reg(weight_matrix):
     #   return 0.01 * np.linalg.norm(np.matrix(np.subtract(np.matmul(weight_matrix,weight_matrix.T,np.matmul(weight_matrix.T,weight_matrix))),'fro')
    model = Sequential()
    model.add(Dense(300, use_bias=False, input_shape=(X_train.shape[1],),kernel_regularizer=nregularizer.l3(l)))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(X_train, y_train, batch_size=128, epochs=20,
                        verbose=False)
    T = model.get_weights()[0]
    
    M = np.matrix(T)

    Fr_norm = np.linalg.norm(np.matrix(np.subtract(np.matmul(M,np.transpose(M)),np.matmul(np.transpose(M),M))),'fro')

    #print ("Determinant:"+str(D))
    
    #print ("Fr_norm:"+str(Fr_norm))

    return T, Fr_norm 

def translation_accuracy(X_test, y_test, T):
    """Get predicted matrix 'yhat' using 'T' and find translation accuracy"""
    # yhat
    yhat = X_test.dot(T)
    count = 0
    for i in range(len(y_test)):
        #if yhat[i,:].all() == y_test[i,:].all():
        if np.array_equal(yhat[i,:],y_test[i,:]) == True:
            count = count + 1
    accuracy = count/len(y_test)*100
    return accuracy

def svd(T):
    """Perform SVD on the translation matrix 'T' """
    U, s, Vh = numpy.linalg.svd(T, full_matrices=False )
    return U, s, Vh

def T_svd_EDA(s):
    """Perform SVD on the translation matrix 'T' """
    plt.hist(s, bins='auto', range = (0,1),normed = 1)
    plt.show()




def normalize(matrix):
    """Normalize the rows of a matrix"""
    matrix_norm = np.linalg.norm(matrix, axis=1)
    matrix_normed = matrix / np.repeat(matrix_norm, matrix.shape[1]). \
        reshape(matrix.shape)
    return matrix_norm, matrix_normed


def translation_results(X, y, vocab, T, lg2_vectors, lg2_vocab):
    """X, y, vocab - The training or test data that you want results for
    T - The translation matrix
    lg2_vectors, lg2_vocab - Foreign language used to find the nearest neighbor
    """

    # Data Prep on Inputs
    X_word, y_word = zip(*vocab)
    X_norm, X_normed = normalize(X)
    y_norm, y_normed = normalize(y)
    lg2_vectors_norm, lg2_vectors_normed = normalize(lg2_vectors)

    # yhat
    yhat = X.dot(T)
    yhat_norm, yhat_normed = normalize(yhat)

    #X_norm = normalize(X)


    # Nearest Neighbors
    neg_cosine = -yhat_normed.dot(lg2_vectors_normed.T)
    ranked_neighbor_indices = np.argsort(neg_cosine, axis=1)

    # Nearest Neighbor
    nearest_neighbor_indices = ranked_neighbor_indices[:, 0]
    yhat_neighbor = lg2_vectors[nearest_neighbor_indices, :]
    yhat_neighbor_norm, yhat_neighbor_normed = normalize(yhat_neighbor)
    yhat_neighbor_word = np.asarray(lg2_vocab)[nearest_neighbor_indices]

    # Results DF
    cols = ['X_norm', 'y_norm', 'yhat_norm', 'yhat_neighbor_norm',
            'X_word', 'y_word', 'yhat_neighbor_word']
    results_df = pd.DataFrame({'X_norm': X_norm,
                               'y_norm': y_norm,
                               'yhat_norm': yhat_norm,
                               'yhat_neighbor_norm': yhat_neighbor_norm,
                               'X_word': X_word,
                               'y_word': y_word,
                               'yhat_neighbor_word': yhat_neighbor_word,})
    results_df = results_df[cols]
    results_df['neighbor_correct'] = results_df.y_word == \
        results_df.yhat_neighbor_word

    return results_df

def T_norm_EDA(results_df):
    """Plot result norms side-by-side"""
    test_size = results_df.shape[0]
    test_accuracy = results_df.neighbor_correct.mean()

    #print('Test Accuracy: '+str(test_accuracy)+'\n')

    #plot_data = ['X_norm', 'y_norm', 'yhat_norm', 'yhat_neighbor_norm']
    # f, ax = plt.subplots(len(plot_data), sharex=True, sharey=True,
    #                      figsize=(10, 10))
    # for i, d in enumerate(plot_data):
    #     ax[i].hist(results_df[d], bins=100)
    #     ax[i].axis('off')
    #     title = '{}: mean={}, std={}'.format(d, round(results_df[d].mean(), 2), round(results_df[d].std(), 2))
    #     ax[i].set_title(title)
    # f.subplots_adjust(hspace=0.7)
    # plt.savefig('../images/' + lg1 + '_' + lg2 + '_' + embedding +
    #             '_T_norm.png')
    # plt.close('all')
    return test_accuracy


def T_pca_EDA(T):
    """PCA on matrix T"""
    T_ss = StandardScaler().fit_transform(T)
    pca = PCA().fit(T_ss)
    n = pca.n_components_

    # plt.figure(figsize=(10, 6))
    # plt.xlim((0, n))
    # plt.ylim((0, 1))
    # plt.plot(range(n + 1), [0] + np.cumsum(pca.explained_variance_ratio_).
    #          tolist())
    # plt.plot(range(n + 1), np.asarray(range(n + 1)) / n)
    # plt.xlabel('Number of Eigenvectors')
    # plt.ylabel('Explained Variance')
    # plt.savefig('../images/' + lg1 + '_' + lg2 + '_' + embedding +
    #             '_T_isotropy.png')
    # plt.close('all')

    isotropy = (1 - sum(np.cumsum(pca.explained_variance_ratio_) * 1 / n)) / .5
    return isotropy


def T_report_results(embedding, lg1, lg2, lg1_vectors, lg2_vectors,
                     X_train, X_test, D, results_df, isotropy):
    md = '## ' + lg1.title() + ' to ' + lg2.title() + ' ' + \
        embedding.title() + '  \n'
    md += '- ' + lg1.title() + ' Vocabulary Size = ' + \
        '{:,.0f}'.format(lg1_vectors.shape[0]) + '  \n'
    md += '- ' + lg1.title() + ' Embedding Length = ' + \
        '{:,.0f}'.format(lg1_vectors.shape[1]) + '  \n'
    md += '- ' + lg2.title() + ' Vocabulary Size = ' + \
        '{:,.0f}'.format(lg2_vectors.shape[0]) + '  \n'
    md += '- ' + lg2.title() + ' Embedding Length = ' + \
        '{:,.0f}'.format(lg2_vectors.shape[1]) + '  \n'
    md += '- Train Size = ' + '{:,.0f}'.format(X_train.shape[0]) + '  \n'
    md += '- Test Size = ' + '{:,.0f}'.format(X_test.shape[0]) + '  \n'
    md += '- Determinant = ' + '{:,.0f}'.format(D) + '  \n'

    md += '- <b>Test Accuracy = ' + \
        '{:,.1%}'.format(results_df.neighbor_correct.mean()) + '</b>  \n\n'



    md += '#### Test L2 Norms  \n'
    md += '- X_norm: L2 norms for ' + lg1.title() + ' test vectors  \n'
    md += '- y_norm: L2 norms for ' + lg2.title() + ' test vectors  \n'
    md += '- yhat_norm: L2 norms for X.dot(T) test vectors ' + \
        '(T = translation matrix)  \n'
    md += '- yhat_neighbor norm: L2 norms for nearest neighbor' + \
        'to X.dot(T) in y test vectors  \n'
    md += '![](../images/' + lg1 + '_' + lg2 + '_' + embedding + \
        '_T_norm.png)  \n\n'

    md += '#### Translation Matrix Isotropy  \n'
    md += '- Isotropy = ' + '{:,.1%}'.format(isotropy) + '  \n'
    md += '![](../images/' + lg1 + '_' + lg2 + '_' + embedding + \
        '_T_isotropy.png)  \n\n'

    return md

### Main Function

In [84]:
if __name__ == '__main__':
    # Manually set list of translations (embedding, lg1, lg2)
    translations = [#('fasttext_random', 'en', 'ru'),
                    ('fasttext_top', 'en', 'en'),
                    ('fasttext_top', 'en', 'ru'),
                    #('fasttext_random', 'en', 'de'),
                    ('fasttext_top', 'en', 'de'),
                    #('fasttext_random', 'en', 'es'),
                    ('fasttext_top', 'en', 'es'),
                    #('fasttext_random', 'en', 'zh-CN'),
                    ('fasttext_top', 'en', 'zh-CN'),
        
        
        
                     #('fasttext_random', 'ru', 'en'),
                    ('fasttext_top', 'ru', 'en'),
                    #('fasttext_random', 'ru', 'es'),
                    ('fasttext_top', 'ru', 'ru'),
                    ('fasttext_top', 'ru', 'de'),
                    ('fasttext_top', 'ru', 'es'),
                    #('fasttext_random', 'ru', 'zh-CN'),
                    ('fasttext_top', 'ru', 'zh-CN'),
                    #('fasttext_random', 'ru', 'de'),
                    
        
        
                    #('fasttext_random', 'de', 'en'),
                    ('fasttext_top', 'de', 'en'),
                    #('fasttext_random', 'de', 'es'),
                    ('fasttext_top', 'de', 'ru'), 
                    #('fasttext_random', 'de', 'ru'),
                    ('fasttext_top', 'de', 'de'),
                    ('fasttext_top', 'de', 'es'),
                    #('fasttext_random', 'de', 'zh-CN'),
                    ('fasttext_top', 'de', 'zh-CN'),
        
        
        
                    #('fasttext_random', 'es', 'en'),
                    ('fasttext_top', 'es', 'en'),
                    #('fasttext_random', 'es', 'de'),
                    ('fasttext_top', 'es', 'ru'),
                    ('fasttext_top', 'es', 'de'),
                    #('fasttext_random', 'es', 'ru'),
                    ('fasttext_top', 'es', 'es'),
                    #('fasttext_random', 'es', 'zh-CN'),
                    ('fasttext_top', 'es', 'zh-CN'),
        
                
        
                    #('fasttext_random', 'zh-CN', 'en'),
                    ('fasttext_top', 'zh-CN', 'en'),
                    #('fasttext_random', 'zh-CN', 'es'),
                    ('fasttext_top', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'de'),
                    ('fasttext_top', 'zh-CN', 'es'),
                    #('fasttext_random', 'zh-CN', 'ru'),
                    ('fasttext_top', 'zh-CN', 'zh-CN'),
                    #('fasttext_random', 'zh-CN', 'de'),
        
                    
                    

                    ]
   
    md = ''
    for translation in translations:
        embedding, lg1, lg2 = translation
        # Vocab/Vectors/Dicts
        lg1_vocab, lg1_vectors, lg2_vocab, lg2_vectors = \
            pickle_rw((lg1 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg1 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vocab', 0),
                      (lg2 + '_' + embedding.split('_')[0] + '_vectors', 0),
                      write=False)
        lg1_dict = make_dict(lg1_vocab, lg1_vectors)
        lg2_dict = make_dict(lg2_vocab, lg2_vectors)

        print('Translation: '+lg1+'->'+lg2+'\n')

        # Train/Test Vocab/Vectors
        vocab_train, vocab_test = vocab_train_test(embedding, lg1, lg2, lg1_vocab)
        X_train, X_test, y_train, y_test = vectors_train_test(vocab_train,
                                                              vocab_test)
 
        
        # Fit tranlation matrix to training data
        
        T, fr_norm = translation_matrix(X_train, y_train, 1) 
        
        
        
        
        
        T1, fr_norm1 = translation_matrix(X_train, y_train, 0.1)
        
        T01, fr_norm01 = translation_matrix(X_train, y_train, 0.01)
        
        T001, fr_norm001 = translation_matrix(X_train, y_train, 0.001)
        
        T0001, fr_norm0001 = translation_matrix(X_train, y_train, 0.0001)
        
        T00001, fr_norm00001 = translation_matrix(X_train, y_train, 0.00001)
        
        # Test Data Results
        
        results_df = translation_results(X_test, y_test, vocab_test, T,
                                         lg2_vectors, lg2_vocab)
        test_accuracy = T_norm_EDA(results_df)
        
#         test_accuracy = translation_accuracy(X_test, y_test, T)
        
        print(test_accuracy)
        
        
        
        results_df1 = translation_results(X_test, y_test, vocab_test, T1,
                                         lg2_vectors, lg2_vocab)
        test_accuracy1 = T_norm_EDA(results_df1)

#         test_accuracy = translation_accuracy(X_test, y_test, T1)
        
        print(test_accuracy1)
        
        
        results_df01 = translation_results(X_test, y_test, vocab_test, T01,
                                         lg2_vectors, lg2_vocab)
        test_accuracy01 = T_norm_EDA(results_df01)
        
        print(test_accuracy01)
        
        
        results_df001 = translation_results(X_test, y_test, vocab_test, T001,
                                         lg2_vectors, lg2_vocab)
        test_accuracy001 = T_norm_EDA(results_df001)
        
        print(test_accuracy001)
        
        
        results_df001 = translation_results(X_test, y_test, vocab_test, T001,
                                         lg2_vectors, lg2_vocab)
        test_accuracy001 = T_norm_EDA(results_df001)
        
        
        results_df0001 = translation_results(X_test, y_test, vocab_test, T0001,
                                         lg2_vectors, lg2_vocab)
        test_accuracy0001 = T_norm_EDA(results_df0001)
        
        
        results_df00001 = translation_results(X_test, y_test, vocab_test, T00001,
                                         lg2_vectors, lg2_vocab)
        test_accuracy00001 = T_norm_EDA(results_df00001)
        
        
        #Plot the line graphs
        
        
        Org_Acc = [test_accuracy ,test_accuracy ,test_accuracy ,test_accuracy, test_accuracy]
        
        New_Acc = []
        New_Acc.append(test_accuracy1)
        New_Acc.append(test_accuracy01)
        New_Acc.append(test_accuracy001)
        New_Acc.append(test_accuracy0001)
        New_Acc.append(test_accuracy00001)
        
        
        #New_Acc = [test_accuracy1 ,test_accuracy01 ,test_accuracy001 ,test_accuracy0001, test_accuracy00001]
        
        Fr_Norm = [fr_norm1, fr_norm01, fr_norm001, fr_norm0001, fr_norm00001]
        
        
#         Org_Acc_df =pd.DataFrame(Org_Acc)
#         Org_Acc_df.columns =['0.1', '0.01', '0.001', '0.0001', '0.00001']
#         New_Acc_df =pd.DataFrame(New_Acc)
#         New_Acc_df.columns =['0.1', '0.01', '0.001', '0.0001', '0.00001']
#         Fr_Norm_df =pd.DataFrame(Fr_norm)
#         Fr_Norm_df.columns =['0.1', '0.01', '0.001', '0.0001', '0.00001']
        
        Org_Acc_df =pd.DataFrame(Org_Acc, index =['0.1', '0.01', '0.001', '0.0001', '0.00001'], columns = ['Org_Acc'], )
        New_Acc_df =pd.DataFrame(New_Acc, index =['0.1', '0.01', '0.001', '0.0001', '0.00001'], columns = ['New_Acc'], )
        Fr_Norm_df =pd.DataFrame(Fr_Norm, index =['0.1', '0.01', '0.001', '0.0001', '0.00001'], columns = ['Fr_Norm'], )

        ax = Org_Acc_df.plot(title=lg1+'->'+lg2+' Accuracies')
        New_Acc_df.plot(ax=ax)
        
        ax = Fr_Norm_df.plot(title=lg1+'->'+lg2+' Frobenius Norm')
        
        
        
        
        


FileNotFoundError: [Errno 2] No such file or directory: 'gs:///pickle/en_fasttext_vocab.pkl'

In [65]:
! ls /root/notebooks



Comparison of Models with Different Lambda Values.ipynb
gensim_download.py
nregularizer.py
__pycache__
PySpark_Comparison of Models with Different Lambda Values.ipynb


In [81]:
! ls /root

notebooks
